In [736]:
from __future__ import division
from UserDict import UserDict
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


# BUILDING NEURAL NETWORKS

### <font color=red>Sample Data</font>

In [737]:
#input sequence
i_1 = [1,0,1,0,1,0,1,0,1]

#forget sequence
f_1 = [0,0,0,0,0,0,0,0,0]

#initialize output
o_1 = [0]

### <font color=red>I/O Nodes</font>

In [740]:
class I_Node(UserDict):
    def __init__(self, name, data):
        UserDict.__init__(self)
        self["name"] = name
        self["data"] = data
        self["counter"] = 0
        self["state"] = self["data"][self["counter"]]
    def update(self):
        self["counter"]= self["counter"]+1
        self["state"] = self["data"][self["counter"]]

In [741]:
class O_Node(UserDict):
    def __init__(self, name, data, inputs=[]):
        UserDict.__init__(self)
        self["name"] = name
        self["data"] = data
        self["state"] = self["data"][-1]
        self["inputs"] = inputs
    def update(self):
        self["state"] = reduce(lambda x, y: x+y, self["inputs"])/len(self["inputs"])
        self["data"].append(self["state"])
        self["inputs"] = []

In [742]:
input_node = I_Node("i_1",i_1)

output_node = O_Node("o_1",o_1)

forget_node = I_Node("f_1",f_1)

### <font color=red>Hidden State Nodes</font>

In [745]:
class Hidden_State(UserDict):
    "reminder: init_state OR input, not both"
    def __init__(self, name, inputs=[], init_state=None):
        UserDict.__init__(self)
        self["name"] = name
        self["inputs"] = inputs
        if inputs:
            self["state"] = reduce(lambda x, y: x+y, self["inputs"])/len(self["inputs"])
        else:
            self["state"] = init_state
    def update(self):
        self["state"]=reduce(lambda x, y: x+y, self["inputs"])/len(self["inputs"])
        self["inputs"] = []

In [764]:
initial_state = 0
hidden_state_1 = Hidden_State("h_1", init_state=initial_state)

**A Little Test**:

In [765]:
hidden_state_1

{'inputs': [], 'state': 0, 'name': 'h_1'}

In [766]:
hidden_state_1["inputs"]=[0,1,1]
hidden_state_1.update()

In [767]:
hidden_state_1

{'inputs': [], 'state': 0.6666666666666666, 'name': 'h_1'}

In [769]:
#back to initial state
hidden_state_1 = Hidden_State("h_1", init_state=initial_state)

### <font color=red>Gates!</font>

In [772]:
#Define opening and closing logic for gates
#for now, we open every 'g' time points
def func(g):
    return input_node["counter"]%g==0

In [770]:
class Gate(UserDict):
    #reminder: add opening and closing logic
    def __init__(self, name, start_node, end_node, state):
        UserDict.__init__(self)
        self["name"] = name
        self["start_node"]= start_node
        self["end_node"]= end_node
        self["state"]= state
        self["func_param"]= func
    def update(self):
        self["state"] = func

In [771]:
input_gate = Gate("g_1",input_node, hidden_state_1,1)

forget_gate = Gate("g_2",forget_node, hidden_state_1,1)

output_gate = Gate("g_3",hidden_state_1,output_node,1)

NameError: global name 'func' is not defined

In [754]:
class Cell(UserDict):
    def __init__(self, name, nodes, gates):
        UserDict.__init__(self)
        self["name"] = name
        self["nodes"] = nodes
        self["gates"] = gates

In [755]:
gates = [forget_gate, input_gate, output_gate]
nodes = [input_node, output_node, forget_node, hidden_state_1]

In [756]:
cell_1 = Cell("cell_1", nodes, gates)

In [757]:
def step(cell):
    #get open gates
    open_gates = filter(lambda x: x["state"], cell_1["gates"])
    #push from node into open gates
    for gate in open_gates:
        end_state = gate["start_node"]["state"]
        start_state = gate["end_node"]["state"]

        #change state
        gate["end_node"]["inputs"].append(end_state)
        
        #summary
        print gate["end_node"]["name"], start_state,"->", gate["end_node"]["inputs"]
    
    #update
    for node in cell_1["nodes"]:
        node.update()
        
    print "======"
    #pull output
    

In [758]:
while 1:
    step(cell_1)

h_1 0.666666666667 -> [0]
h_1 0.666666666667 -> [0, 1]
o_1 0 -> [0.6666666666666666]
h_1 0.5 -> [0]
h_1 0.5 -> [0, 0]
o_1 0.666666666667 -> [0.5]
h_1 0.0 -> [0]
h_1 0.0 -> [0, 1]
o_1 0.5 -> [0.0]
h_1 0.5 -> [0]
h_1 0.5 -> [0, 0]
o_1 0.0 -> [0.5]
h_1 0.0 -> [0]
h_1 0.0 -> [0, 0]
o_1 0.5 -> [0.0]
h_1 0.0 -> [0]
h_1 0.0 -> [0, 0]
o_1 0.0 -> [0.0]
h_1 0.0 -> [0]
h_1 0.0 -> [0, 1]
o_1 0.0 -> [0.0]
h_1 0.5 -> [0]
h_1 0.5 -> [0, 1]
o_1 0.0 -> [0.5]
h_1 0.5 -> [1]
h_1 0.5 -> [1, 1]
o_1 0.5 -> [0.5]


IndexError: list index out of range

In [759]:
input_node

{'state': 1, 'counter': 9, 'data': [1, 0, 1, 0, 0, 0, 1, 1, 1], 'name': 'i_1'}

In [760]:
hidden_state_1

{'inputs': [1, 1], 'state': 0.5, 'name': 'h_1'}

In [761]:
output_node["data"]

[0, 0.6666666666666666, 0.5, 0.0, 0.5, 0.0, 0.0, 0.0, 0.5]

In [762]:
len(output_node["data"])

9

In [730]:
#reset input counters
for node in cell_1["nodes"]:
    if isinstance(node, I_Node):
        node["counter"] = 0